# DF

In [ ]:
import marimo as mo
import polars as pl

from r_translation import join

matching, contentious = join()
matching = matching.with_columns(
    genus=pl.when(
        (pl.col("genus").is_null()) & (pl.col("specificEpithet").is_not_null())
    )
    .then(pl.col("taxonName").str.split(" ").list[0])
    .otherwise("genus")
)

bos_df columns: ['speciesId', 'phylum', 'class', 'subclass', 'order', 'sub-order', 'infraorder', 'section', 'subsection', 'superfamily', 'family', 'subfamily', 'tribe', 'genus', 'subgenus', 'specificEpithet', 'infraspecificEpithet', 'taxonName', 'source_file', 'taxon', 'superorder', 'domain', 'kingdom', 'section/parvorder', 'Column2', 'infraclass', 'commonName', 'species2', 'clade 1', 'clade 2', 'clade 3', 'genericName']


taxon.csv columns, odict_keys(['taxonID', 'genericName', 'genus', 'specificEpithet', 'infraspecificEpithet', 'taxonomicStatus', 'acceptedNameUsageID'])


In [ ]:
contentious.collect_schema().keys()

odict_keys(['speciesId', 'matched_taxonID', 'acceptedNameUsageID', 'taxonName', 'domain', 'kingdom', 'phylum', 'class', 'subclass', 'superorder', 'order', 'sub-order', 'infraorder', 'section', 'subsection', 'superfamily', 'family', 'subfamily', 'tribe', 'genus', 'subgenus', 'genericName', 'specificEpithet', 'infraspecificEpithet'])

In [ ]:
# matching = df.filter(~pl.col("speciesId").is_duplicated()).with_columns(
#     acceptedNameUsageID=pl.col("acceptedNameUsageID")
#     .fill_null(pl.lit(-1))
#     .cast(pl.Int64)
# )
# contentious = df.filter((pl.col("speciesId").is_duplicated())).with_columns(
#     acceptedNameUsageID=pl.col("acceptedNameUsageID")
#     .fill_null(pl.lit(-1))
#     .cast(pl.Int64)
# )

# unique_contentious = contentious.filter(
#     (pl.col("acceptedNameUsageID") == -1)
#     & (
#         pl.col("matched_taxonID").is_in(
#             pl.col("acceptedNameUsageID").implode()
#         )
#     )
# )
# # print(
# #     unique_contentious.select(
# #         "speciesId", "acceptedNameUsageID", "matched_taxonID"
# #     ).collect()
# # )
# unique_contentius_speciesId = (
#     unique_contentious.select("speciesId").collect().to_series().implode()
# )  # Just the speciesIds
# contentious2 = contentious.filter(
#     ~pl.col("speciesId").is_in(unique_contentius_speciesId)
# )  # Removing...

# # print(contentious2.select('speciesId','acceptedNameUsageID','matched_taxonID').collect())

# matching = pl.concat(
#     [matching, unique_contentious],
# )

nomatch lazyframe is for BOS data point that has no current match in gbif data set.

In [ ]:
debug = (
    matching.filter(
        (pl.col("specificEpithet").is_not_null()) & (pl.col("genus").is_null())
    )
    .select(
        [
            "speciesId",
            "taxonName",
            "infraspecificEpithet",
            "specificEpithet",
            "genus",
            "family",
            "order",
            "class",
            "phylum",
            "kingdom",
        ]
    )
    .collect()
)
debug
# matching.filter(
#     (pl.col("specificEpithet").is_not_null())
#     & (
#         pl.col("speciesId").is_in(
#             debug.select(["speciesId"]).to_series().to_list()
#         )
#     )
# ).select(
#     [
#         "taxonName",
#     ]
#     + priorityFeatures
# ).collect()

speciesId,taxonName,infraspecificEpithet,specificEpithet,genus,family,order,class,phylum,kingdom
str,str,str,str,str,str,str,str,str,str


In [ ]:
nomatch = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .fill_null("")  # TODO TEST IF NOT WORK, REMOVE
    .collect()
)
nomatch

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,kingdom,phylum,class,subclass,superorder,order,sub-order,infraorder,section,subsection,superfamily,family,subfamily,tribe,genus,subgenus,genericName,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
"""A-Anne-Clit-Hirudinea-000001""",null,null,"""Glossiphoniidae sp. ""1""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Rhynchobdellida""","""""","""""","""""","""""","""""","""Glossiphoniidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000002""",null,null,"""Glossiphoniidae sp. ""2""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Rhynchobdellida""","""""","""""","""""","""""","""""","""Glossiphoniidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000003""",null,null,"""Glossiphoniidae sp. ""3""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Rhynchobdellida""","""""","""""","""""","""""","""""","""Glossiphoniidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000004""",null,null,"""Hirudinidae sp. ""1""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Arhynchobdellida""","""""","""""","""""","""""","""""","""Hirudinidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000005""",null,null,"""Hirudinidae sp. ""2""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Arhynchobdellida""","""""","""""","""""","""""","""""","""Hirudinidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A-Hemi-Enteropneusta-000010""",null,null,"""Acorn worm sp.""","""Eukarya""","""Animalia""","""Hemichordata""","""Enteropneusta""","""""","""""","""blank-UnID""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Hemi-Enteropneusta-000011""",null,null,"""Acorn worm sp.""","""Eukarya""","""Animalia""","""Hemichordata""","""Enteropneusta""","""""","""""","""blank-UnID""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Phoronida-000002""",null,null,"""Phoronida sp.""","""Eukarya""","""Animalia""","""Phoronida""","""blank-Phor""","""""","""""","""blank-Phor""","""""","""""","""""","""""","""""","""Phoronidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null


In [ ]:
matching_with_populated_match_taxonID = matching.filter(
    pl.col("matched_taxonID").is_not_null()
).with_columns(
    taxonRank=pl.lit(None),
    taxonomicStatus=pl.lit(None),
    parentNameUsageID=pl.lit(None),
)

# repeated_accepted_taxons = RAT

In [ ]:
repeated_accepted_taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
    .filter(pl.col("canonicalName").is_duplicated())
)

In [ ]:
priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
]

_schema = {
    "feature_that_is_equal_to_canonicalName": pl.String,
    "matches": pl.String,
}
_r = (repeated_accepted_taxons).collect()
RAT_interim = pl.DataFrame(schema=_schema)
for _c in priority_columns:
    _a = (
        _r.filter(pl.col("canonicalName") == pl.col(_c))
        .select("canonicalName")
        .unique()
        # .collect()
    )
    if _a.shape[0] != 0:
        _t = _a["canonicalName"].to_list()

        _row = pl.DataFrame(
            data={"feature_that_is_equal_to_canonicalName": _c, "matches": _t},
            schema=_schema,
        )
        RAT_interim = RAT_interim.vstack(_row)
RAT_feats = (
    RAT_interim.group_by("matches")
    .agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))
    .with_columns(
        pl.col("feature_that_is_equal_to_canonicalName").str.split(", ")
    )
    .sort("feature_that_is_equal_to_canonicalName")
)
# RAT_feats.write_csv("RAT_feats.csv")

In [ ]:
taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(
        pl.col("taxonomicStatus") == pl.lit("accepted"),
        pl.col("taxonRank") != "unranked",
    )
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .select(["taxonID"] + priority_columns)
)
taxons.collect().shape

In [ ]:
RAT_feats.filter(pl.col("matches") == "Tentaculata")

matches,feature_that_is_equal_to_canonicalName
str,list[str]


# matching attempt

### Still_no_match is not comprehensive, doesn't includes subset  with some matches and some without

In [ ]:
updated_to_matching = []
all_taxon_data_to_be_selected_from = []
still_no_match = []
_collected_repeated_taxons = repeated_accepted_taxons.collect().fill_null("")
_reversed_priority_columns = priority_columns.copy()
_reversed_priority_columns.reverse()
for _f in _reversed_priority_columns:
    to_skip = False
    for _m in RAT_feats["matches"]:
        # skipping those that are not in RAT_feats
        if _m not in nomatch[_f].to_list():
            continue

        print("-------\nFeature:", _f, ",Name:", _m)

        taxon_data_to_select_from = _collected_repeated_taxons.filter(
            pl.col("canonicalName") == _m
        ).select(["taxonID"] + _reversed_priority_columns)

        # Select from the two rows what taxon data to select.
        selected_row = -1
        for i, _t in enumerate(
            taxon_data_to_select_from.select(
                ["taxonID"] + priority_columns
            ).iter_rows()
        ):
            col_index = priority_columns.index(_f)
            _x = _t[col_index]
            if not bool(_x):
                selected_row = i  # Since there are only two rows from taxon_data_to_select_from, the results is either 0 or 1
                break
        assert selected_row != -1

        # has_predicament1 is when kingdom to genus all has values and it is kind of leveled between the two rows of the taxon data.
        has_predicament1 = False
        _l = taxon_data_to_select_from["family"].to_list()
        if _f == "genus" and _l[0] != None and _l[1] != None:
            print("same level detected")
            has_predicament1 = True

        chosen_taxonId = taxon_data_to_select_from[selected_row, 0]
        other_taxonId = taxon_data_to_select_from[
            int(not bool(selected_row)), 0
        ]

        # Getting the _no_match_subset_to_update section
        _no_match_subset_to_update = nomatch.filter(
            pl.col(_f) == _m,
        )

        _temp2 = _reversed_priority_columns[:-3]
        features_to_find_nulls = (
            _temp2[_reversed_priority_columns.index(_f) + 1]
            if len(_temp2) > _reversed_priority_columns.index(_f) + 1
            else None
        )
        if features_to_find_nulls is not None:
            print("features_to_find_nulls", features_to_find_nulls)
            _no_match_subset_to_update = _no_match_subset_to_update.filter(
                pl.col(features_to_find_nulls) == ""
            )

            if _no_match_subset_to_update.is_empty():
                # If not empty parent id will still be filled below
                still_no_match_subset = nomatch.filter(
                    pl.col(_f) == _m,
                ).with_columns(
                    current_feature=pl.lit(_f), current_name=pl.lit(_m)
                )
                still_no_match.append(still_no_match_subset)
                print(
                    "\n\n\n\n\nskipping",
                    _f,
                    _m,
                    "\n this _no_match_subset_to_update dataframe doesn't have the same tax rank level\n",
                    still_no_match_subset.select(
                        # [
                        #     "parentNameUsageID",
                        #     "taxonName",
                        # ]
                        # +
                        _reversed_priority_columns
                    ),
                )
                # print(
                #     "taxon_data_to_select_from\n",
                #     taxon_data_to_select_from,
                # )
                print("---------")
                continue

        # Settleing predicament one
        # this predicament thing should only happen when the subset is of only 1 row.
        if has_predicament1 and _no_match_subset_to_update.shape[0] == 1:
            match = True
            for _f1 in ["class", "order", "family"]:
                match *= (
                    _no_match_subset_to_update[_f1].item()
                    == taxon_data_to_select_from[selected_row, :][_f1].item()
                )
            if not match:
                match = True
                for _f1 in ["class", "order", "family"]:
                    match *= (
                        _no_match_subset_to_update[_f1].item()
                        == taxon_data_to_select_from[
                            int(not bool(selected_row)), :
                        ][_f1].item()
                    )
                assert match == True
                print("changing chosen taxonId")
                temp = chosen_taxonId
                chosen_taxonId = other_taxonId
                other_taxonId = temp

        # turning these rows to matching by filling parentNameUsageID
        _no_match_subset_to_update = _no_match_subset_to_update.with_columns(
            parentNameUsageID=pl.when(
                (pl.col("infraspecificEpithet").is_null())
                & (pl.col("specificEpithet").is_not_null())
                & (pl.col("genus").is_not_null())
            )
            .then(pl.lit(other_taxonId))
            .otherwise(pl.lit(chosen_taxonId))
        )

        # Debugging for matching data from gbif to new matched rows
        print(
            "taxon_data_to_select_from\n",
            taxon_data_to_select_from,
        )
        print("parentNameUsageID chosen:", chosen_taxonId)
        print("no_match_subset_to_update ")
        print(
            _no_match_subset_to_update.select(
                # [
                #     "parentNameUsageID",
                #     "taxonName",
                # ]
                # +
                _reversed_priority_columns
            )
        )
        print("---------")

        updated_to_matching.append(_no_match_subset_to_update)
        all_taxon_data_to_be_selected_from.append(taxon_data_to_select_from)

# turn them into dataframes and writing to csv files
updated_to_matching = pl.concat(
    updated_to_matching, rechunk=True, parallel=True
)
updated_to_matching.write_csv("updated_to_matching.csv")
all_taxon_data_to_be_selected_from = pl.concat(
    all_taxon_data_to_be_selected_from, rechunk=True, parallel=True
)
all_taxon_data_to_be_selected_from.write_csv(
    "all_taxon_data_to_be_selected_from.csv"
)
still_no_match = pl.concat(still_no_match, rechunk=True, parallel=True)
still_no_match.write_csv("still_no_match.csv")

-------
Feature: phylum ,Name: Arthropoda


features_to_find_nulls class





skipping phylum Arthropoda 
 this _no_match_subset_to_update dataframe doesn't have the same tax rank level
 

shape: (2_266, 8)
┌──────────┬────────────┬─────────────┬─────────────┬────────────┬───────┬────────────┬────────────┐
│ kingdom  ┆ phylum     ┆ class       ┆ order       ┆ family     ┆ genus ┆ specificEp ┆ infraspeci │
│ ---      ┆ ---        ┆ ---         ┆ ---         ┆ ---        ┆ ---   ┆ ithet      ┆ ficEpithet │
│ str      ┆ str        ┆ str         ┆ str         ┆ str        ┆ str   ┆ ---        ┆ ---        │
│          ┆            ┆             ┆             ┆            ┆       ┆ str        ┆ str        │
╞══════════╪════════════╪═════════════╪═════════════╪════════════╪═══════╪════════════╪════════════╡
│ Animalia ┆ Arthropoda ┆ Arachnida   ┆ Amblypygi   ┆            ┆       ┆            ┆            │
│ Animalia ┆ Arthropoda ┆ Arachnida   ┆ Araneae     ┆ Araneidae  ┆       ┆            ┆            │
│ Animalia ┆ Arthropoda ┆ Arachnida   ┆ Araneae     ┆ Hahniidae  ┆       ┆            ┆            │
│ Animalia ┆ Arthropoda ┆ Arachnida   ┆ Araneae     ┆ Salticidae ┆       






skipping phylum Ctenophora 
 this _no_match_subset_to_update dataframe doesn't have the same tax rank level
 shape: (11, 8)
┌──────────┬────────────┬─────────────┬─────────────┬────────────┬───────┬────────────┬────────────┐
│ kingdom  ┆ phylum     ┆ class       ┆ order       ┆ family     ┆ genus ┆ specificEp ┆ infraspeci │
│ ---      ┆ ---        ┆ ---         ┆ ---         ┆ ---        ┆ ---   ┆ ithet      ┆ ficEpithet │
│ str      ┆ str        ┆ str         ┆ str         ┆ str        ┆ str   ┆ ---        ┆ ---        │
│          ┆            ┆             ┆             ┆            ┆       ┆ str        ┆ str        │
╞══════════╪════════════╪═════════════╪═════════════╪════════════╪═══════╪════════════╪════════════╡
│ Animalia ┆ Ctenophora ┆ Tentaculata ┆ Platyctenid ┆ Coeloplani ┆       ┆            ┆            │
│          ┆            ┆             ┆ a           ┆ dae        ┆       ┆            ┆            │
│ Animalia ┆ Ctenophora ┆ Tentaculata ┆ Platyctenid ┆ Coeloplan

-------
Feature: class ,Name: Polychaeta
features_to_find_nulls order


...

-------
Feature: family ,Name: Sigalionidae
taxon_data_to_select_from
 shape: (2, 9)
┌──────────┬──────────┬──────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ taxonID  ┆ kingdom  ┆ phylum   ┆ class      ┆ … ┆ family     ┆ genus     ┆ specificE ┆ infraspec │
│ ---      ┆ ---      ┆ ---      ┆ ---        ┆   ┆ ---        ┆ ---       ┆ pithet    ┆ ificEpith │
│ i64      ┆ str      ┆ str      ┆ str        ┆   ┆ str        ┆ str       ┆ ---       ┆ et        │
│          ┆          ┆          ┆            ┆   ┆            ┆           ┆ str       ┆ ---       │
│          ┆          ┆          ┆            ┆   ┆            ┆           ┆           ┆ str       │
╞══════════╪══════════╪══════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 12188730 ┆ Animalia ┆ Annelida ┆ Polychaeta ┆ … ┆ Sigalionid ┆ Sigalioni ┆           ┆           │
│          ┆          ┆          ┆            ┆   ┆ ae         ┆ dae       ┆           ┆           │
│ 7081

taxon_data_to_select_from
 shape: (2, 9)
┌──────────┬──────────┬────────────┬─────────┬───┬────────┬────────┬───────────────┬───────────────┐
│ taxonID  ┆ kingdom  ┆ phylum     ┆ class   ┆ … ┆ family ┆ genus  ┆ specificEpith ┆ infraspecific │
│ ---      ┆ ---      ┆ ---        ┆ ---     ┆   ┆ ---    ┆ ---    ┆ et            ┆ Epithet       │
│ i64      ┆ str      ┆ str        ┆ str     ┆   ┆ str    ┆ str    ┆ ---           ┆ ---           │
│          ┆          ┆            ┆         ┆   ┆        ┆        ┆ str           ┆ str           │
╞══════════╪══════════╪════════════╪═════════╪═══╪════════╪════════╪═══════════════╪═══════════════╡
│ 4334     ┆ Animalia ┆ Arthropoda ┆ Insecta ┆ … ┆ Apidae ┆        ┆               ┆               │
│ 10501510 ┆ Animalia ┆ Arthropoda ┆ Insecta ┆ … ┆ Apidae ┆ Apidae ┆               ┆               │
└──────────┴──────────┴────────────┴─────────┴───┴────────┴────────┴───────────────┴───────────────┘
parentNameUsageID chosen: 4334
no_match_subset_to_

-------
Feature: genus ,Name: Polyura
same level detected
changing chosen taxonId
taxon_data_to_select_from
 shape: (2, 9)
┌─────────┬──────────┬────────────┬────────────┬───┬────────────┬─────────┬────────────┬───────────┐
│ taxonID ┆ kingdom  ┆ phylum     ┆ class      ┆ … ┆ family     ┆ genus   ┆ specificEp ┆ infraspec │
│ ---     ┆ ---      ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---     ┆ ithet      ┆ ificEpith │
│ i64     ┆ str      ┆ str        ┆ str        ┆   ┆ str        ┆ str     ┆ ---        ┆ et        │
│         ┆          ┆            ┆            ┆   ┆            ┆         ┆ str        ┆ ---       │
│         ┆          ┆            ┆            ┆   ┆            ┆         ┆            ┆ str       │
╞═════════╪══════════╪════════════╪════════════╪═══╪════════════╪═════════╪════════════╪═══════════╡
│ 2900801 ┆ Plantae  ┆ Tracheophy ┆ Magnoliops ┆ … ┆ Rubiaceae  ┆ Polyura ┆            ┆           │
│         ┆          ┆ ta         ┆ ida        ┆   ┆            ┆    

-------
Feature: genus ,Name: Lutzia
same level detected
changing chosen taxonId
taxon_data_to_select_from
 shape: (2, 9)
┌─────────┬──────────┬────────────┬────────────┬───┬────────────┬────────┬────────────┬────────────┐
│ taxonID ┆ kingdom  ┆ phylum     ┆ class      ┆ … ┆ family     ┆ genus  ┆ specificEp ┆ infraspeci │
│ ---     ┆ ---      ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---    ┆ ithet      ┆ ficEpithet │
│ i64     ┆ str      ┆ str        ┆ str        ┆   ┆ str        ┆ str    ┆ ---        ┆ ---        │
│         ┆          ┆            ┆            ┆   ┆            ┆        ┆ str        ┆ str        │
╞═════════╪══════════╪════════════╪════════════╪═══╪════════════╪════════╪════════════╪════════════╡
│ 7292005 ┆ Plantae  ┆ Tracheophy ┆ Magnoliops ┆ … ┆ Brassicace ┆ Lutzia ┆            ┆            │
│         ┆          ┆ ta         ┆ ida        ┆   ┆ ae         ┆        ┆            ┆            │
│ 1650076 ┆ Animalia ┆ Arthropoda ┆ Insecta    ┆ … ┆ Culicidae  ┆ Lutz

-------
Feature: genus ,Name: Dryas
same level detected
taxon_data_to_select_from
 shape: (2, 9)
┌─────────┬──────────┬─────────────┬────────────┬───┬────────────┬───────┬────────────┬────────────┐
│ taxonID ┆ kingdom  ┆ phylum      ┆ class      ┆ … ┆ family     ┆ genus ┆ specificEp ┆ infraspeci │
│ ---     ┆ ---      ┆ ---         ┆ ---        ┆   ┆ ---        ┆ ---   ┆ ithet      ┆ ficEpithet │
│ i64     ┆ str      ┆ str         ┆ str        ┆   ┆ str        ┆ str   ┆ ---        ┆ ---        │
│         ┆          ┆             ┆            ┆   ┆            ┆       ┆ str        ┆ str        │
╞═════════╪══════════╪═════════════╪════════════╪═══╪════════════╪═══════╪════════════╪════════════╡
│ 1893309 ┆ Animalia ┆ Arthropoda  ┆ Insecta    ┆ … ┆ Nymphalida ┆ Dryas ┆            ┆            │
│         ┆          ┆             ┆            ┆   ┆ e          ┆       ┆            ┆            │
│ 3000739 ┆ Plantae  ┆ Tracheophyt ┆ Magnoliops ┆ … ┆ Rosaceae   ┆ Dryas ┆            ┆        

there is rows that are totally skipped and those that are partially skipped or i call it as left out..

In [ ]:
matching2 = pl.concat(
    [updated_to_matching, matching_with_populated_match_taxonID.collect()]
)
matching2.write_csv("matching2.csv")

# still no match stuff

## `current_feature` uniques

In [ ]:
still_no_match["current_feature"].unique()

current_feature
str
"""order"""
"""phylum"""


## `current_name` uniques

In [ ]:
still_no_match["current_name"].unique()

current_name
str
"""Arthropoda"""
"""Passeriformes"""
"""Ctenophora"""
"""Siluriformes"""


## number of rows in still no match

In [ ]:
still_no_match.shape[0]

2287

# Testing  new `Taxons` df with `still_no_match`

## stop at class, there is only current_feature = phylum matches

In [ ]:
## stop at class, there is only current_feature = phylum matches
match_on_class = still_no_match.with_columns(
    name_to_match=pl.when(
        pl.col("order") == "",
        pl.col("family") == "",
        pl.col("class").is_not_null(),
        pl.col("current_feature") == "phylum",
    )
    .then("class")
    .otherwise(None)
).filter(
    pl.col("order") == "",
    pl.col("family") == "",
    pl.col("class").is_not_null(),
    pl.col("current_feature") == "phylum",
)

_x1 = (
    still_no_match.filter(
        pl.col("order") == "",
        pl.col("family") == "",
        pl.col("class").is_not_null(),
        pl.col("current_feature") == "phylum",
    )
    .select("class", "phylum")
    .group_by("class", "phylum")
    .len()
)

_taxons_subset = taxons.filter(
    pl.col("family").is_null(),
    pl.col("order").is_null(),
    pl.col("genus").is_null(),
    pl.col("class").is_in(_x1["class"].unique()),
    pl.col("specificEpithet").is_null(),
    pl.col("infraspecificEpithet").is_null(),
    pl.col("phylum").is_in(_x1["phylum"].unique()),
).select("taxonID", "class")
match_on_class = (
    pl.LazyFrame(match_on_class)
    .join(
        _taxons_subset,
        on="class",
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
).collect()

/tmp/marimo_128664/__marimo__cell_nHfw_.py:52: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  ).collect()


In [ ]:
match_on_class.group_by("parentNameUsageID", "name_to_match").len()

parentNameUsageID,name_to_match,len
i64,str,u32
10713444,"""Collembola""",2
365,"""Tentaculata""",7
11545536,"""Copepoda""",17
353,"""Ostracoda""",16


In [ ]:
match_on_class.shape

## stops at order there is only current_feature = phylum matches

In [ ]:
## stops at order there is only current_feature = phylum matches


match_on_order = still_no_match.with_columns(
    name_to_match=pl.when(
        pl.col("order") != "",
        pl.col("family") == "",
        pl.col("current_feature") == "phylum",
    )
    .then("order")
    .otherwise(None)
).filter(
    pl.col("order") != "",
    pl.col("family") == "",
    # pl.col("class").is_not_null(),
    pl.col("current_feature") == "phylum",
)
# print(match_on_order.select('order'))
_x1 = (
    still_no_match.filter(
        pl.col("order") != "",
        pl.col("family") == "",
        # pl.col("class").is_not_null(),
        pl.col("current_feature") == "phylum",
    )
    .select("order", "phylum")
    .group_by("order", "phylum")
    .len()
)
# print(_x1)

_taxons_subset = (
    taxons.filter(
        pl.col("family").is_null(),
        pl.col("order").is_in(_x1["order"].unique().to_list()),
        pl.col("genus").is_null(),
        # pl.col("class").is_in(_x1["class"].unique()),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
        pl.col("phylum").is_in(_x1["phylum"].unique().to_list()),
    )
    .select("taxonID", "order")
    .collect()
)
match_on_order = (
    match_on_order.join(
        _taxons_subset,
        on="order",
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)

In [ ]:
match_on_order.group_by("parentNameUsageID", "name_to_match").len()

parentNameUsageID,name_to_match,len
i64,str,u32
1500,"""Cumacea""",6
787,"""Plecoptera""",1
697,"""Pedunculata""",4
1224,"""Dermaptera""",2
1373,"""Stomatopoda""",10
…,…,…
9477065,"""Pantopoda""",19
841,"""Mysida""",7
788,"""Mantodea""",2


In [ ]:
match_on_order.shape

## stops at family, current_feature contains both phylum and order

In [ ]:
## stops at family there is only current_feature = phylum matches


match_on_family = still_no_match.with_columns(
    name_to_match=pl.when(
        pl.col("family") != "",
        # pl.col("current_feature") == "phylum",
    )
    .then("family")
    .otherwise(None)
).filter(
    pl.col("family") != "",
    # pl.col("class").is_not_null(),
    # pl.col("current_feature") == "phylum",
)
# print(match_on_family.select('family'))
_x1 = (
    still_no_match.filter(
        pl.col("family") != "",
        # pl.col("current_feature") == "phylum",
    )
    .select("family", "phylum", "order")
    .group_by("family", "phylum", "order")
    .len()
)
# print(_x1)

_taxons_subset = (
    taxons.filter(
        pl.col("family").is_in(_x1["family"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
        pl.col("phylum").is_in(_x1["phylum"].unique().to_list()),
        pl.col("order").is_in(_x1["order"].unique().to_list()),
    )
    .select("taxonID", "family")
    .collect()
)
match_on_family = (
    match_on_family.join(
        _taxons_subset,
        on="family",
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)

In [ ]:
match_on_family.group_by(
    "parentNameUsageID", "name_to_match", "current_feature"
).len().filter(pl.col("current_feature") == "phylum")

parentNameUsageID,name_to_match,current_feature,len
i64,str,str,u32
7824,"""Dytiscidae""","""phylum""",1
5030,"""Sacculinidae""","""phylum""",1
4355,"""Termitidae""","""phylum""",16
7366,"""Desidae""","""phylum""",1
4304235,"""Maxillipiidae""","""phylum""",1
…,…,…,…
9650,"""Pentatomidae""","""phylum""",4
3862,"""Diogenidae""","""phylum""",14
3040,"""Tetrigidae""","""phylum""",7


In [ ]:
match_on_family.group_by(
    "parentNameUsageID", "name_to_match", "current_feature"
).len().filter(pl.col("current_feature") == "order")

parentNameUsageID,name_to_match,current_feature,len
i64,str,str,u32
3241609,"""Timaliidae""","""order""",1
4512,"""Ariidae""","""order""",3
9228,"""Pangasiidae""","""order""",1
4287160,"""Dicaeidae""","""order""",1
5164,"""Plotosidae""","""order""",4


In [ ]:
match_on_family.shape

## stops at specificEpithet there is only current_feature = phylum matches

#TODO: SOMETHING IS WRONG WITH THIS, SPECIFICEPITHET AND GENUS FOR BOTH PHYLUM AND ORDER CURRENT_FEATURE CAN"T BE FOUDN IN GBIF

In [ ]:
## stops at family there is only current_feature = phylum matches


match_on_genus = still_no_match.with_columns(
    name_to_match=pl.when(
        pl.col("specificEpithet") != "",
        pl.col("genus") != "",
    )
    .then(
        pl.concat_str(pl.col("specificEpithet"), pl.lit(" "), pl.col("genus"))
    )
    .otherwise(None)
).filter(
    pl.col("specificEpithet") != "",
    pl.col("genus") != "",
)

# print(match_on_genus['current_feature'].unique().to_list())
_x1 = (
    still_no_match.filter(
        pl.col("specificEpithet") != "",
        pl.col("genus") != "",
    )
    .select(
        "specificEpithet",
        # "phylum",
        "genus",
    )
    .group_by(
        "specificEpithet",
        # "phylum",
        "genus",
    )
    .len()
)


# This statement below seems to be wrong
print("Glaucias" in _x1["genus"].unique().to_list())
print("dorsalis" in _x1["specificEpithet"].unique().to_list())
_taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    # .filter(
    #     pl.col("taxonomicStatus") == pl.lit("accepted"),
    #     pl.col("taxonRank") != "unranked",
    # )
    # .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    # .select(["taxonID"] + priority_columns)
)
_taxons_subset = (
    _taxons.filter(
        pl.col("specificEpithet").is_in(
            _x1["specificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(_x1["genus"].unique().to_list()),
        # pl.col("phylum").is_in(_x1["phylum"].unique().to_list()),
        # pl.col("infraspecificEpithet").is_null(),
    )
    .select(
        "taxonID",
        "specificEpithet",
        "genus",
    )
    .collect()
)
print(
    match_on_genus.filter(pl.col("current_feature") == "phylum").select(
        "specificEpithet", "genus", "phylum"
    )
)
print(
    _taxons_subset.filter(
        pl.col("genus") == "Glaucias", pl.col("specificEpithet") == "dorsalis"
    )
)
print(
    _taxons.filter(
        pl.col("genus") == "Glaucias", pl.col("specificEpithet") == "dorsalis"
    ).collect()
)

# TAXON.TSV doesn't have pl.col("genus")=='Dicaeum',pl.col('specificEpithet')=='chysorrheum'
match_on_genus = (
    match_on_genus.join(_taxons_subset, on=["genus", "specificEpithet"])
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)
print(match_on_genus.filter(pl.col("genus") == "Glaucias"))

True
True


shape: (208, 3)
┌─────────────────┬───────────────┬────────────┐
│ specificEpithet ┆ genus         ┆ phylum     │
│ ---             ┆ ---           ┆ ---        │
│ str             ┆ str           ┆ str        │
╞═════════════════╪═══════════════╪════════════╡
│ tumidus         ┆ Enoplolambrus ┆ Arthropoda │
│ pheromosa       ┆ Nocticola     ┆ Arthropoda │
│ nirgriceps      ┆ Perigona      ┆ Arthropoda │
│ vomeratus       ┆ Amphisternus  ┆ Arthropoda │
│ quadrimacula    ┆ Episcapha     ┆ Arthropoda │
│ …               ┆ …             ┆ …          │
│ bucculatus      ┆ Aeschines     ┆ Arthropoda │
│ trimaculatus    ┆ Paracritheus  ┆ Arthropoda │
│ valbum          ┆ Elasmolomus   ┆ Arthropoda │
│ dorsalis        ┆ Glaucias      ┆ Arthropoda │
│ warwickii       ┆ Octolasmis    ┆ Arthropoda │
└─────────────────┴───────────────┴────────────┘
shape: (0, 3)
┌─────────┬─────────────────┬───────┐
│ taxonID ┆ specificEpithet ┆ genus │
│ ---     ┆ ---             ┆ ---   │
│ i64     ┆ str        

shape: (0, 23)
┌─────────┬───────────┬───────────────────┬───────────────────┬───┬───────┬───────┬────────┬───────┐
│ taxonID ┆ datasetID ┆ parentNameUsageID ┆ acceptedNameUsage ┆ … ┆ class ┆ order ┆ family ┆ genus │
│ ---     ┆ ---       ┆ ---               ┆ ID                ┆   ┆ ---   ┆ ---   ┆ ---    ┆ ---   │
│ i64     ┆ str       ┆ i64               ┆ ---               ┆   ┆ str   ┆ str   ┆ str    ┆ str   │
│         ┆           ┆                   ┆ str               ┆   ┆       ┆       ┆        ┆       │
╞═════════╪═══════════╪═══════════════════╪═══════════════════╪═══╪═══════╪═══════╪════════╪═══════╡
└─────────┴───────────┴───────────────────┴───────────────────┴───┴───────┴───────┴────────┴───────┘
shape: (0, 30)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ speciesId ┆ matched_t ┆ acceptedN ┆ taxonName ┆ … ┆ parentNam ┆ current_f ┆ current_n ┆ name_to_ │
│ ---       ┆ axonID    ┆ ameUsageI ┆ ---       ┆   ┆ eUsageI

#combining updated parent id rows

In [ ]:
_new_match = pl.concat([match_on_class, match_on_order, match_on_family]).drop(
    ["current_feature", "current_name", "name_to_match"]
)
_new_match = pl.concat(
    [
        _new_match,
        updated_to_matching.with_columns(
            pl.col("parentNameUsageID").cast(pl.Int64)
        ),
    ]
)
_new_match.write_csv("updated_to_matching.csv")
_new_match

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,kingdom,phylum,class,subclass,superorder,order,sub-order,infraorder,section,subsection,superfamily,family,subfamily,tribe,genus,subgenus,genericName,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""A-Arth-Crus-Copepoda-000005""",null,null,"""Copepod sp. 1""","""Eukarya""","""Animalia""","""Arthropoda""","""Copepoda""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",11545536
"""A-Arth-Crus-Copepoda-000006""",null,null,"""Copepod sp. 2""","""Eukarya""","""Animalia""","""Arthropoda""","""Copepoda""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",11545536
"""A-Arth-Crus-Copepoda-000007""",null,null,"""Copepod sp. 3""","""Eukarya""","""Animalia""","""Arthropoda""","""Copepoda""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",11545536
"""A-Arth-Crus-Copepoda-000008""",null,null,"""Copepod sp. 4""","""Eukarya""","""Animalia""","""Arthropoda""","""Copepoda""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",11545536
"""A-Arth-Crus-Copepoda-000009""",null,null,"""Copepod sp. 5""","""Eukarya""","""Animalia""","""Arthropoda""","""Copepoda""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",11545536
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A-Anne-Polychaeta-000490""",null,null,"""Halosydna? sp.""","""Eukarya""","""Animalia""","""Annelida""","""Polychaeta""","""Errantia""","""""","""Phyllodocida""","""""","""""","""""","""""","""""","""Polynoidae""","""""","""""","""""","""""","""Halosydna""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",7078
"""A-Anne-Polychaeta-000497""",null,null,"""Polynoidae sp.""","""Eukarya""","""Animalia""","""Annelida""","""Polychaeta""","""Errantia""","""""","""Phyllodocida""","""""","""""","""""","""""","""""","""Polynoidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",7078
"""A-Arth-Hexa-Lepidoptera-000192""",null,null,"""Polyura schreiber tisamenus""","""Eukarya""","""Animalia""","""""","""Insecta""","""""","""""","""Lepidoptera""","""""","""""","""""","""""","""""","""Nymphalidae""","""Charaxinae""","""""","""Polyura""","""""","""Polyura""","""schreiber""","""tisamenus""","""BOSuncornirmedSpecies""","""BOSunformired""",1892952


# OLD

## stop at class, there is only current_feature = phylum matches

## stops at order there is only current_feature = phylum matches

## stops at family, current_feature = phylum

## Stops at one of the epithet, current_feature contains = phylum

In [ ]:
debug1 = still_no_match.select(
    [
        "current_feature",
        "current_name",
        # "parentNameUsageID",
        "taxonName",
    ]
    + priority_columns
)

debug1.filter(pl.col("order") == "", pl.col("current_feature") == "phylum")

current_feature,current_name,taxonName,infraspecificEpithet,specificEpithet,genus,family,order,class,phylum,kingdom
str,str,str,str,str,str,str,str,str,str,str
"""phylum""","""Arthropoda""","""Copepod sp. 1""","""""","""""","""""","""""","""""","""Copepoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Copepod sp. 2""","""""","""""","""""","""""","""""","""Copepoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Copepod sp. 3""","""""","""""","""""","""""","""""","""Copepoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Copepod sp. 4""","""""","""""","""""","""""","""""","""Copepoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Copepod sp. 5""","""""","""""","""""","""""","""""","""Copepoda""","""Arthropoda""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""phylum""","""Ctenophora""","""Comb jelly sp. 3""","""""","""""","""""","""""","""""","""Tentaculata""","""Ctenophora""","""Animalia"""
"""phylum""","""Ctenophora""","""Comb jelly sp. 4""","""""","""""","""""","""""","""""","""Tentaculata""","""Ctenophora""","""Animalia"""
"""phylum""","""Ctenophora""","""Comb jelly sp. 5""","""""","""""","""""","""""","""""","""Tentaculata""","""Ctenophora""","""Animalia"""


### class names and phylum names:

In [ ]:
debug1.filter(
    pl.col("order") == "", pl.col("current_feature") == "phylum"
).select("class", "phylum").group_by("class", "phylum").len()

class,phylum,len
str,str,u32
"""Tentaculata""","""Ctenophora""",7
"""Collembola""","""Arthropoda""",2
"""Ostracoda""","""Arthropoda""",16
"""Merostomata""","""Arthropoda""",5
"""Copepoda""","""Arthropoda""",17


potential matches from gbif.org  but not in `repeated_accepted_taxons`

|class|phylumn|taxonID from gbif.org|url|
|:---|:----|:----|:--
|Ostracoda|Arthropoda|353|https://www.gbif.org/species/353|

In [ ]:
debug1.filter(
    pl.col("family") == "",
    pl.col("order") != "",
    pl.col("genus") == "",
    pl.col("current_feature") == "phylum",
)

current_feature,current_name,taxonName,infraspecificEpithet,specificEpithet,genus,family,order,class,phylum,kingdom
str,str,str,str,str,str,str,str,str,str,str
"""phylum""","""Arthropoda""","""Amblypygi sp. ZRC_ENT00014159""","""""","""""","""""","""""","""Amblypygi""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Araneae sp. ZRC_BDP0072018""","""""","""""","""""","""""","""Araneae""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Opiliones sp. ZRCBDP_0071973""","""""","""""","""""","""""","""Opiliones""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Scorpiones sp. ZRCBDP_0071969""","""""","""""","""""","""""","""Scorpiones""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Scorpiones sp. ZRCBDP_0072011""","""""","""""","""""","""""","""Scorpiones""","""Arachnida""","""Arthropoda""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""phylum""","""Arthropoda""","""Barnacle sp. SS-4392""","""""","""""","""""","""""","""Sessilia""","""Maxillopoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Barnacle sp. SUB-1448""","""""","""""","""""","""""","""Sessilia""","""Maxillopoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Barnacle sp. SEA-8136""","""""","""""","""""","""""","""Sessilia""","""Maxillopoda""","""Arthropoda""","""Animalia"""


### order names when current name = Arthropoda:

In [ ]:
# current Name:
debug1.filter(
    pl.col("family") == "",
    pl.col("order") != "",
    pl.col("genus") == "",
    pl.col("current_feature") == "phylum",
)["current_name"].unique().to_list()

In [ ]:
debug1.filter(
    pl.col("family") == "",
    pl.col("order") != "",
    pl.col("genus") == "",
    pl.col("current_feature") == "phylum",
)["order"].unique().to_list()

In [ ]:
debug1.filter(
    pl.col("family") != "",
    pl.col("genus") == "",
    pl.col("current_feature") == "phylum",
)

current_feature,current_name,taxonName,infraspecificEpithet,specificEpithet,genus,family,order,class,phylum,kingdom
str,str,str,str,str,str,str,str,str,str,str
"""phylum""","""Arthropoda""","""Araneidae sp. ""ZRC_BDP0050080""""","""""","""""","""""","""Araneidae""","""Araneae""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Hahniidae sp. ""ZRC_BDP0050902""""","""""","""""","""""","""Hahniidae""","""Araneae""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Salticidae sp. ""ZRC_BDP0057873…","""""","""""","""""","""Salticidae""","""Araneae""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Myrmarachne sp. ""ZRC_BDP006085…","""""","""""","""""","""Salticidae""","""Araneae""","""Arachnida""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Rhene sp. ""ZRC_BDP0060870""""","""""","""""","""""","""Salticidae""","""Araneae""","""Arachnida""","""Arthropoda""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""phylum""","""Arthropoda""","""Thompsonia sp.""","""""","""""","""""","""Thompsoniidae""","""Akentrogonida""","""Maxillopoda""","""Arthropoda""","""Animalia"""
"""phylum""","""Ctenophora""","""Coeloplana sp. 1""","""""","""""","""""","""Coeloplanidae""","""Platyctenida""","""Tentaculata""","""Ctenophora""","""Animalia"""
"""phylum""","""Ctenophora""","""Coeloplana sp. 2""","""""","""""","""""","""Coeloplanidae""","""Platyctenida""","""Tentaculata""","""Ctenophora""","""Animalia"""


### family names where current_feature = phylum

In [ ]:
debug1.filter(
    pl.col("family") != "",
    pl.col("genus") == "",
    pl.col("current_feature") == "phylum",
)["family"].unique().to_list()

### current names where current_feature = phylum:

In [ ]:
debug1.filter(
    pl.col("family") != "",
    pl.col("genus") == "",
    pl.col("current_feature") == "phylum",
)["current_name"].unique().to_list()

In [ ]:
debug1.filter(pl.col("genus") != "", pl.col("current_feature") == "phylum")

current_feature,current_name,taxonName,infraspecificEpithet,specificEpithet,genus,family,order,class,phylum,kingdom
str,str,str,str,str,str,str,str,str,str,str
"""phylum""","""Arthropoda""","""Enoplolambrus tumidus""","""""","""tumidus""","""Enoplolambrus""","""Parthenopidae""","""Decapoda""","""Malacostraca""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Nocticola pheromosa""","""""","""pheromosa""","""Nocticola""","""Nocticolidae""","""Blattodea""","""Insecta""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Perigona nigriceps""","""""","""nirgriceps""","""Perigona""","""Carabidae""","""Coleoptera""","""Insecta""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Amphisternus vomeratus""","""""","""vomeratus""","""Amphisternus""","""Endomychidae""","""Coleoptera""","""Insecta""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Episcapha quadrimacula""","""""","""quadrimacula""","""Episcapha""","""Erotylidae""","""Coleoptera""","""Insecta""","""Arthropoda""","""Animalia"""
…,…,…,…,…,…,…,…,…,…,…
"""phylum""","""Arthropoda""","""Aeschines bucculatus""","""""","""bucculatus""","""Aeschines""","""Pyrrhocoridae""","""Hemiptera""","""Insecta""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Paracritheus trimaculatus""","""""","""trimaculatus""","""Paracritheus""","""Pentatomidae""","""Hemiptera""","""Insecta""","""Arthropoda""","""Animalia"""
"""phylum""","""Arthropoda""","""Elasmolomus valbum""","""""","""valbum""","""Elasmolomus""","""Rhyparochromidae""","""Hemiptera""","""Insecta""","""Arthropoda""","""Animalia"""


### specificEpithet names when current_feature = phylum and current_name = "Arthropoda"

#### current_name when current_feature = phylum

In [ ]:
debug1.filter(pl.col("genus") != "", pl.col("current_feature") == "phylum")[
    "current_name"
].unique().to_list()

In [ ]:
debug1.filter(pl.col("genus") != "", pl.col("current_feature") == "phylum")[
    "specificEpithet"
].unique().to_list()